# Data Preprocessing

## Library

In [27]:
import pandas as pd
import re

## Load data

In [28]:
train = pd.read_csv('../data/train.csv')
dev = pd.read_csv('../data/dev.csv')
test = pd.read_csv('../data/test.csv')

## Preprocessing

### 좌우 공백 제거
### 자음 or 모음만으로 구성된 경우 적절한 값으로 대체

In [29]:
train2 = train.copy()
dev2 = dev.copy()
test2 = test.copy()

In [30]:
print(train2.dialogue.apply(lambda x: len(x)).mean())
print(train2.summary.apply(lambda x: len(x)).mean())
print(dev2.dialogue.apply(lambda x: len(x)).mean())
print(dev2.summary.apply(lambda x: len(x)).mean())
print(test2.dialogue.apply(lambda x: len(x)).mean())

438.76511198522917
87.39792887533113
432.55911823647295
81.70941883767534
449.312625250501


In [31]:
## train ##

# 좌우 공백 제거
train['dialogue'] = train['dialogue'].apply(lambda x: x.strip())
train['summary'] = train['summary'].apply(lambda x: x.strip())
train['topic'] = train['topic'].apply(lambda x: x.strip())

# 자음, 모음만으로 구성된 경우 적절한 값으로 대체
train['dialogue'] = train['dialogue'].apply(lambda x: x.replace('제ㅏ', '제가'))
train['dialogue'] = train['dialogue'].apply(lambda x: x.replace('척했ㄷ거든', '척했거든'))
train['dialogue'] = train['dialogue'].apply(lambda x: x.replace('배경ㅇ로', '배경으로'))
train['dialogue'] = train['dialogue'].apply(lambda x: x.replace('ㅋㅋ', '웃기다'))
train['dialogue'] = train['dialogue'].apply(lambda x: x.replace('아직ㅍ알맞는', '아직 알맞는'))
train['summary'] = train['summary'].apply(lambda x: x.replace('머라이어 ㅐ리', '머라이어 캐리'))


## dev ##

# 좌우 공백 제거
dev['dialogue'] = dev['dialogue'].apply(lambda x: x.strip())
dev['summary'] = dev['summary'].apply(lambda x: x.strip())
dev['topic'] = dev['topic'].apply(lambda x: x.strip())

## test ##

# 좌우 공백 제거
test['dialogue'] = test['dialogue'].apply(lambda x: x.strip())

In [32]:
print(train2.dialogue.apply(lambda x: len(x)).mean() - train.dialogue.apply(lambda x: len(x)).mean())
print(train2.summary.apply(lambda x: len(x)).mean() - train.summary.apply(lambda x: len(x)).mean())
print(dev2.dialogue.apply(lambda x: len(x)).mean() - dev.dialogue.apply(lambda x: len(x)).mean())
print(dev2.summary.apply(lambda x: len(x)).mean() - dev.summary.apply(lambda x: len(x)).mean())
print(test2.dialogue.apply(lambda x: len(x)).mean() - test.dialogue.apply(lambda x: len(x)).mean())

0.034277916031101086
0.008428995745362045
0.05210420841683572
0.008016032064119827
0.04008016032065598


공백을 제거하자 length가 감소하는 것을 알 수 있다.

#### train special tokens 처리

In [33]:
hash_list_1 = []
hash_list_2 = []

for i in range(len(train)):
    hash_list_1.extend(re.findall("#(.*?)#", train.iloc[i, 1]))
    hash_list_2.extend(re.findall("#(.*?)#", train.iloc[i, 2]))

In [34]:
set(hash_list_1)

{'',
 'Address',
 'CarNumber',
 'CardNumber',
 'DateOfBirth',
 'Email',
 'PassportNumber',
 'Person',
 'Person 2',
 'Person1',
 'Person2',
 'Person3',
 'Person4',
 'Person5',
 'Person6',
 'Person7',
 'PhoneNumber',
 'PhoneNumber이고, 휴대폰 번호는 ',
 'SSN'}

In [35]:
set(hash_list_2)

{'Person1',
 'Person2',
 'Person3',
 'Person4',
 'Person6',
 'Person7',
 '에게 내일 아침 7시와 7시 30분에 ',
 '은 그 기간 동안 ',
 '은 시험에 대한 준비가 된 것 같다며 ',
 '이 '}

In [36]:
# 찾고자 하는 해시태그
search_tags = ['', 'Person', 'Person 2',
               'PhoneNumber이고, 휴대폰 번호는 ']

# 발견된 행 번호를 저장할 리스트
found_rows = []

for i in range(len(train)):
    # 1번째 열에서 해시태그 찾기
    hashtags_1 = re.findall("#(.*?)#", train.iloc[i, 1])
    # 2번째 열에서 해시태그 찾기
    hashtags_2 = re.findall("#(.*?)#", train.iloc[i, 2])
    
    # 빈 문자열이나 'Person'이 포함되어 있는지 확인
    if any(tag in search_tags for tag in hashtags_1) or any(tag in search_tags for tag in hashtags_2):
        # 조건을 만족하는 경우 행 번호 저장
        found_rows.append(i)

# 결과 출력
print("Found rows:", found_rows)

Found rows: [420, 1899, 4537, 11713]


In [37]:
train.iloc[420, 1] = re.sub(r'(?<!\#)#PhoneNumber(?!\#)', '#PhoneNumber#', train.iloc[420, 1])
train.iloc[1899, 1] = re.sub(r'#Person#', '#Person2#', train.iloc[1899, 1])
train.iloc[4537, 1] = re.sub(r'#Person 2#', '#Person2#', train.iloc[4537, 1])
train.iloc[11713, 1] = re.sub(r'##Person1#', '#Person1#', train.iloc[11713, 1])

In [38]:
# 찾고자 하는 해시태그
search_tags = ['에게 내일 아침 7시와 7시 30분에 ',
               '은 그 기간 동안 ',
               '은 시험에 대한 준비가 된 것 같다며 ',
               '이 ']

# 발견된 행 번호를 저장할 리스트
found_rows = []

for i in range(len(train)):
    # 1번째 열에서 해시태그 찾기
    hashtags_1 = re.findall("#(.*?)#", train.iloc[i, 1])
    # 2번째 열에서 해시태그 찾기
    hashtags_2 = re.findall("#(.*?)#", train.iloc[i, 2])
    
    # 빈 문자열이나 'Person'이 포함되어 있는지 확인
    if any(tag in search_tags for tag in hashtags_1) or any(tag in search_tags for tag in hashtags_2):
        # 조건을 만족하는 경우 행 번호 저장
        found_rows.append(i)

# 결과 출력
print("Found rows:", found_rows)

Found rows: [1030, 1133, 1199]


In [39]:
train.iloc[1030, 2] = re.sub(r'사람2#', '#Person2#', train.iloc[1030, 2])
train.iloc[1133, 2] = re.sub(r'사람2#', '#Person2#', train.iloc[1133, 2])
train.iloc[1199, 2] = re.sub(r'사람1#', '#Person1#', train.iloc[1199, 2])

#### dev special tokens 처리

In [40]:
hash_list_3 = []
hash_list_4 = []

for i in range(len(dev)):
    hash_list_3.extend(re.findall("#(.*?)#", dev.iloc[i, 1]))
    hash_list_4.extend(re.findall("#(.*?)#", dev.iloc[i, 2]))

In [41]:
set(hash_list_3) # dialogue는 괜찮음

{'Address',
 'DateOfBirth',
 'Person1',
 'Person2',
 'Person3',
 'Person4',
 'PhoneNumber'}

In [42]:
set(hash_list_4) # summary는 수정 필요함

{'Person1', 'Person2', 'Person3', '사람1'}

In [43]:
# 찾고자 하는 해시태그
search_tags = ['사람1']

# 발견된 행 번호를 저장할 리스트
found_rows = []

for i in range(len(dev)):
    # 1번째 열에서 해시태그 찾기
    hashtags_3 = re.findall("#(.*?)#", dev.iloc[i, 1])
    # 2번째 열에서 해시태그 찾기
    hashtags_4 = re.findall("#(.*?)#", dev.iloc[i, 2])
    
    # 빈 문자열이나 'Person'이 포함되어 있는지 확인
    if any(tag in search_tags for tag in hashtags_3) or any(tag in search_tags for tag in hashtags_4):
        # 조건을 만족하는 경우 행 번호 저장
        found_rows.append(i)

# 결과 출력
print("Found rows:", found_rows)

Found rows: [43]


In [44]:
dev.iloc[43, 2] = re.sub(r'#사람1#', '#Person1#', dev.iloc[43, 2])

#### test는 special tokens가 잘 처리 되어 있었음

In [45]:
test

,fname,dialogue
0,test_0,"#Person1#: 더슨 씨, 받아쓰기 좀 해주세요. \n#Person2#: 네, ..."
1,test_1,#Person1#: 드디어 왔네! 왜 그렇게 오래 걸렸어?\n#Person2#: 또...
2,test_2,"#Person1#: 케이트, 무슨 일이 일어났는지 너는 믿지 못할거야. \n#Per..."
3,test_3,"#Person1#: 생일 축하해, 이건 너를 위한 거야, 브라이언.\n#Person..."
4,test_4,#Person1#: 이 올림픽 공원이 정말 크네요!\n#Person2#: 네. 지금...
...,...,...
494,test_495,"#Person1#: 헤이, 찰리, 학교 끝나고 우리 집에 와서 나랑 비디오 게임 할..."
495,test_496,#Person1#: 어떻게 컨트리 음악에 관심을 가지게 되었나요?\r\n#Perso...
496,test_497,"#Person1#: 실례합니다, 앨리스. 이곳을 사용해본 적이 없는데, 기계를 어떻..."
497,test_498,#Person1#: 매튜? 안녕!\r\n#Person2#: 스티브! 오랜만이네! 얼...


In [46]:
hash_list_5 = []

for i in range(len(dev)):
    hash_list_5.extend(re.findall("#(.*?)#", test.iloc[i, 1]))

In [47]:
set(hash_list_5)

{'Address', 'PassportNumber', 'Person1', 'Person2', 'Person3', 'PhoneNumber'}

In [48]:
train.to_csv('../data/train.csv', index=False)
dev.to_csv('../data/dev.csv', index=False)
test.to_csv('../data/test.csv', index=False)